In [1]:
import pandas as pd
import numpy as np
import time
import yfinance as yf

In [2]:
TICKS = [
    'SPY',
    'EFA',
    'EEM',
    'PSP',
    'QAI',
    'HYG',
    'DBC',
    'IYR',
    'IEF',
    'BWX',
    'TIP',
    'SHV',
]

FLDS = ['shortName','quoteType','currency','volume','totalAssets','longBusinessSummary']

In [3]:
info = pd.DataFrame(index=TICKS,columns=FLDS)
info.index.name = 'ticker'
for tick in info.index:
    temp = yf.Ticker(tick).get_info()

    for fld in FLDS:
        if fld in temp.keys():
            info.loc[tick,fld] = temp[fld]
    
    #time.sleep(0.5)

In [4]:
info

,shortName,quoteType,currency,volume,totalAssets,longBusinessSummary
ticker,,,,,,
SPY,SPDR S&P 500,ETF,USD,24710239,603516960768,The trust seeks to achieve its investment obje...
EFA,iShares MSCI EAFE ETF,ETF,USD,4818965,63209721856,The fund generally will invest at least 80% of...
EEM,iShares MSCI Emerging Index Fun,ETF,USD,8563600,17162833920,The fund generally will invest at least 80% of...
PSP,Invesco Global Listed Private E,ETF,USD,3622,258060880,The fund generally will invest at least 90% of...
QAI,NYLI Hedge Multi-Strategy Track,ETF,USD,12559,696944896,"The fund is a ""fund of funds"" which means it i..."
HYG,iShares iBoxx $ High Yield Corp,ETF,USD,19724563,15944028160,The underlying index is a rules-based index co...
DBC,Invesco DB Commodity Index Trac,ETF,USD,184605,1142769792,The fund pursues its investment objective by i...
IYR,iShares U.S. Real Estate ETF,ETF,USD,2170005,3480166144,The fund seeks to track the investment results...
IEF,iShares 7-10 Year Treasury Bond,ETF,USD,5269754,34938195968,The underlying index measures the performance ...


In [5]:
STARTDATE = '2011-01-01'
ENDDATE = '2025-05-31'

tickers = list(info.index.values)
df = yf.download(tickers, start=STARTDATE, end=ENDDATE)['Close']

/var/folders/zx/3v_qt0957xzg3nqtnkv007d00000gn/T/ipykernel_31936/1766415481.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(tickers, start=STARTDATE, end=ENDDATE)['Close']
[*********************100%***********************]  12 of 12 completed


In [6]:
prices = df.resample('M').last()

# eliminate timezones
prices.index = prices.index.tz_localize(None)

rets = prices.pct_change().dropna()

# change to excess returns, in excess of short-term treasury
retsx = rets.subtract(rets['SHV'], axis=0)
retsx = retsx.drop(columns=['SHV'])

/var/folders/zx/3v_qt0957xzg3nqtnkv007d00000gn/T/ipykernel_31936/3831157974.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  prices = df.resample('M').last()


In [7]:
rets

Ticker,BWX,DBC,EEM,EFA,HYG,IEF,IYR,PSP,QAI,SHV,SPY,TIP
Date,,,,,,,,,,,,
2011-02-28,0.006617,0.041710,-0.000437,0.035498,0.014354,-0.002083,0.045204,0.040146,0.002583,0.000091,0.034737,0.007208
2011-03-31,0.008427,0.026582,0.062896,-0.023883,0.000424,-0.001546,-0.010935,0.016667,0.005521,0.000172,0.000120,0.011903
2011-04-30,0.048805,0.045559,0.027327,0.056258,0.015976,0.018412,0.046633,0.058671,0.019034,0.000545,0.028961,0.023780
2011-05-31,-0.013545,-0.051724,-0.029400,-0.022061,0.001333,0.025056,0.010134,-0.041565,0.000000,-0.000100,-0.011214,0.002659
2011-06-30,-0.000097,-0.042645,-0.009355,-0.012108,-0.005706,-0.005045,-0.031060,-0.042581,-0.010776,0.000172,-0.016869,0.007549
...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-31,0.000935,0.027596,0.021521,0.048009,0.013605,0.006165,0.018913,0.067995,0.016237,0.003633,0.026856,0.013702
2025-02-28,0.013991,0.001366,0.011470,0.029531,0.009703,0.027997,0.037861,-0.040107,-0.005952,0.003115,-0.012695,0.021572
2025-03-31,0.010353,0.022727,0.011340,0.001839,-0.010858,0.003420,-0.023382,-0.060826,-0.013237,0.003343,-0.055719,0.006797


In [8]:
retsx

Ticker,BWX,DBC,EEM,EFA,HYG,IEF,IYR,PSP,QAI,SPY,TIP
Date,,,,,,,,,,,
2011-02-28,0.006526,0.041620,-0.000528,0.035407,0.014264,-0.002174,0.045113,0.040055,0.002492,0.034647,0.007117
2011-03-31,0.008256,0.026410,0.062725,-0.024055,0.000252,-0.001717,-0.011107,0.016495,0.005349,-0.000052,0.011731
2011-04-30,0.048260,0.045014,0.026782,0.055713,0.015431,0.017867,0.046088,0.058126,0.018489,0.028416,0.023235
2011-05-31,-0.013446,-0.051624,-0.029300,-0.021961,0.001433,0.025156,0.010233,-0.041465,0.000100,-0.011115,0.002759
2011-06-30,-0.000270,-0.042817,-0.009527,-0.012280,-0.005878,-0.005217,-0.031232,-0.042753,-0.010948,-0.017042,0.007377
...,...,...,...,...,...,...,...,...,...,...,...
2025-01-31,-0.002698,0.023963,0.017888,0.044377,0.009972,0.002533,0.015280,0.064362,0.012604,0.023224,0.010070
2025-02-28,0.010876,-0.001750,0.008355,0.026416,0.006587,0.024882,0.034746,-0.043222,-0.009067,-0.015810,0.018457
2025-03-31,0.007010,0.019385,0.007997,-0.001504,-0.014201,0.000077,-0.026725,-0.064169,-0.016579,-0.059062,0.003454


In [9]:
with pd.ExcelWriter('../data/multi_asset_etf_data.xlsx') as writer:  
    info.to_excel(writer, sheet_name= 'descriptions')
    prices.to_excel(writer, sheet_name= 'prices')
    rets.to_excel(writer, sheet_name='total returns')
    retsx.to_excel(writer, sheet_name='excess returns')